In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)

import numpy as np

import plotly.express as px
import plotly.subplots as sp

from load import *

# Load data

In [3]:
EXPERIMENT_PATH = "../results/2022-01-24_200857/"

In [ ]:
df, sampled_df = load_data(EXPERIMENT_PATH)

# Heatmap

In [6]:
# Get a single, exemplary run to visualize node distribution
heatmap_df = df[
    (df["Nodes"] == 1000)
    & (df["Mode"] == "SF9, 125kHz, 51B")
    & (df["Messages/Node"] == 3)
    & (df["Current Seed"] == 35039)
]

In [36]:
# Fig: The nodes' spatial distribution
fig = px.density_heatmap(
    heatmap_df,
    x="Sender Position X",
    y="Sender Position Y",
    range_color=[0,1],
    labels={
        'Sender Position X': 'Position (m)',
        'Sender Position Y': 'Position (m)',
    }
)

fig.update_layout(
    coloraxis_showscale=False,
    yaxis = {
     'tickmode': 'array',
     'tickvals': [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000],
     'ticktext': ["0", "1000", "2000", "3000", "4000", "5000", "6000", "7000", "8000", "9000", "10000"],
    },
    xaxis = {
     'tickmode': 'array',
     'tickvals': [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000],
     'ticktext': ["0", "1000", "2000", "3000", "4000", "5000", "6000", "7000", "8000", "9000", "10000"],
    }
)
fig.write_image("node_locations.pdf", width=700, height=700)

# Prepare relative event counts

In [35]:
rx_counts = df[df["Event"] == "RX"].groupby(["Current Seed", "Nodes", "Mode", 'Messages/Node'], dropna=True).size().reset_index(name='count').set_index(["Current Seed", "Nodes", "Mode", 'Messages/Node'], drop=False)
tx_counts = df[df["Event"] == "TX"].groupby(["Current Seed", "Nodes", "Mode", 'Messages/Node'], dropna=True).size().reset_index(name='count').set_index(["Current Seed", "Nodes", "Mode", 'Messages/Node'], drop=False)
fi_counts = df[df["Event"] == "FI"].groupby(["Current Seed", "Nodes", "Mode", 'Messages/Node'], dropna=True).size().reset_index(name='count').set_index(["Current Seed", "Nodes", "Mode", 'Messages/Node'], drop=False)
fs_counts = df[df["Event"] == "FS"].groupby(["Current Seed", "Nodes", "Mode", 'Messages/Node'], dropna=True).size().reset_index(name='count').set_index(["Current Seed", "Nodes", "Mode", 'Messages/Node'], drop=False)
fw_counts = df[df["Event"] == "FW"].groupby(["Current Seed", "Nodes", "Mode", 'Messages/Node'], dropna=True).size().reset_index(name='count').set_index(["Current Seed", "Nodes", "Mode", 'Messages/Node'], drop=False)

counts = rx_counts.join(tx_counts, lsuffix="_RX", rsuffix="_TX")
counts.drop(["Current Seed_RX", 'Nodes_RX', 'Mode_RX', 'Messages/Node_RX'], axis='columns', inplace=True)
counts.rename({
    "Current Seed_TX": "Current Seed",
    'Nodes_TX': 'Nodes',
    'Mode_TX': 'Mode',
    'Messages/Node_TX': 'Messages/Node'}, axis='columns', inplace=True)
counts["Success"] = (rx_counts["count"] / ((tx_counts["Nodes"] * tx_counts["Messages/Node"]) * (tx_counts["Nodes"] - 1))) * 100

counts = counts.join(fi_counts, lsuffix="_cnt", rsuffix="_FI")
counts.drop(["Current Seed_cnt", 'Nodes_cnt', 'Mode_cnt', 'Messages/Node_cnt'], axis='columns', inplace=True)
counts.rename({
    "Current Seed_FI": "Current Seed",
    'Nodes_FI': 'Nodes',
    'Mode_FI': 'Mode',
    'Messages/Node_FI': 'Messages/Node'}, axis='columns', inplace=True)
counts["Failure (Interference)"] = (fi_counts["count"] / ((tx_counts["Nodes"] * tx_counts["Messages/Node"]) * (tx_counts["Nodes"] - 1))) * 100

counts = counts.join(fs_counts, lsuffix="_cnt", rsuffix="_FS")
counts.drop(["Current Seed_cnt", 'Nodes_cnt', 'Mode_cnt', 'Messages/Node_cnt'], axis='columns', inplace=True)
counts.rename({
    "Current Seed_FS": "Current Seed",
    'Nodes_FS': 'Nodes',
    'Mode_FS': 'Mode',
    'Messages/Node_FS': 'Messages/Node'}, axis='columns', inplace=True)
counts["Failure (Signal Strength)"] = (fs_counts["count"] / ((tx_counts["Nodes"] * tx_counts["Messages/Node"]) * (tx_counts["Nodes"] - 1))) * 100

counts = counts.join(fw_counts, lsuffix="_cnt", rsuffix="_FW")
counts.drop(["Current Seed_cnt", 'Nodes_cnt', 'Mode_cnt', 'Messages/Node_cnt'], axis='columns', inplace=True)
counts.rename({
    "Current Seed_FW": "Current Seed",
    'Nodes_FW': 'Nodes',
    'Mode_FW': 'Mode',
    'Messages/Node_FW': 'Messages/Node'}, axis='columns', inplace=True)
counts["Failure (Invalid Receiver State)"] = (fw_counts["count"] / ((tx_counts["Nodes"] * tx_counts["Messages/Node"]) * (tx_counts["Nodes"] - 1))) * 100

counts.rename(
    {"count_cnt": "count_FI", 'count': 'count_FW'},
    axis='columns',
    inplace=True
)

values = {
    "Failure (Interference)": 0.0,
    "Failure (Invalid Receiver State)": 0.0,
    "count_FI": 0.0,
    "count_FW": 0.0,
}

counts.fillna(value=values, inplace=True)

counts.drop(["Current Seed", "Nodes", "Mode", "Messages/Node"], axis='columns', inplace=True)
counts.reset_index(inplace=True)

counts["Failure (Invalid Sender State)"] = 100 \
    - counts["Failure (Invalid Receiver State)"] \
    - counts["Failure (Signal Strength)"] \
    - counts["Failure (Interference)"] \
    - counts["Success"]

counts["count_FT"] = ((counts["Nodes"] * counts["Messages/Node"]) - counts["count_TX"]) * (counts["Nodes"] - 1)

# Frame states (realtive)

In [37]:
# Fig: Ratio of frame states
# Use only runs from the regular experiments (i.e., 3, 10 and 50 messages/node) and do not show TX events
smaller_count = counts[
    (~counts["Messages/Node"].isin([1, 20, 30, 40, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]))
    & (counts["Mode"] != "SF9, 125kHz, 115B")
]

fig = px.histogram(
    smaller_count,
    x="Mode",
    y=["Success", "Failure (Signal Strength)", "Failure (Interference)", "Failure (Invalid Receiver State)", "Failure (Invalid Sender State)"],
    facet_col="Nodes",
    facet_row="Messages/Node",
    histfunc='avg',
    color_discrete_sequence=px.colors.qualitative.Plotly,
    category_orders={
        "Mode": ["SF7, 250kHz, 222B", "SF7, 125kHz, 222B", "SF7, 125kHz, 51B", "SF9, 125kHz, 51B", "SF12, 125kHz, 51B"],
        "Nodes": [100, 500, 1000],
        "Messages/Node": [50, 10, 3]
    },
    labels={
        "Mode": "Configuration",
        "Messages/Node": "Messages per Node",
        "variable": "Packet State",
    },
    color_discrete_map={
        "Success": px.colors.qualitative.Plotly[0],
        "Failure (Signal Strength)": px.colors.qualitative.Plotly[1],
        "Failure (Interference)": px.colors.qualitative.Plotly[2],
        "Failure (Invalid Receiver State)": px.colors.qualitative.Plotly[3],
        "Failure (Invalid Sender State)": px.colors.qualitative.Plotly[4]
    },
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="center",
    x=.5
))

fig.write_image("frame_states.pdf", width=1100, height=618)

# Frame states (absolute)

In [38]:
# Fig: Absolute numbers of frame states
# Only 10 messages/node and 500 nodes
smaller_count = counts[
    (counts["Messages/Node"].isin([10]))
    & (counts["Mode"] != "SF9, 125kHz, 115B")
    & (counts["Nodes"] == 500)
]

fig = px.histogram(
    smaller_count,
    x="Mode", 
    y=["count_RX", "count_FS", "count_FI", "count_FW", "count_FT"],
    histfunc='avg',
    category_orders={
        "Mode": ["SF7, 250kHz, 222B", "SF7, 125kHz, 222B", "SF7, 125kHz, 51B", "SF9, 125kHz, 51B", "SF12, 125kHz, 51B"],
    },
    labels={
        "Mode": "Configuration",
        "variable": "Packet State"
    },
    color_discrete_map={
        "count_RX": px.colors.qualitative.Plotly[0],
        "count_FS": px.colors.qualitative.Plotly[1],
        "count_FI": px.colors.qualitative.Plotly[2],
        "count_FW": px.colors.qualitative.Plotly[3],
        "count_FT": px.colors.qualitative.Plotly[4]
    },
)

newnames = {
    'count_RX': 'Success',
    'count_FS': 'Failure (Signal Strength)',
    "count_FI": "Failure (Interference)",
    "count_FW": "Failure (Invalid Receiver State)",
    "count_FT": "Failure (Invalid Sender State)"
}

fig.for_each_trace(lambda t: t.update(name=newnames[t.name], legendgroup=newnames[t.name], hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name])))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="center",
    x=.5
))

fig.write_image("receive_states_absolute_10_500.pdf", width=1100, height=618)

In [39]:
# Fig: Absolute numbers of frame states
# Only 50 messages/node and 1000 nodes
smaller_count = counts[
    (counts["Messages/Node"].isin([50]))
    & (counts["Mode"] != "SF9, 125kHz, 115B")
    & (counts["Nodes"] == 1000)
]

fig = px.histogram(
    smaller_count,
    x="Mode", 
    y=["count_RX", "count_FS", "count_FI", "count_FW", "count_FT"],
    histfunc='avg',
    category_orders={
        "Mode": ["SF7, 250kHz, 222B", "SF7, 125kHz, 222B", "SF7, 125kHz, 51B", "SF9, 125kHz, 51B", "SF12, 125kHz, 51B"],
    },
    labels={
        "Mode": "Configuration",
        "variable": "Packet State"
    },
    color_discrete_map={
        "count_RX": px.colors.qualitative.Plotly[0],
        "count_FS": px.colors.qualitative.Plotly[1],
        "count_FI": px.colors.qualitative.Plotly[2],
        "count_FW": px.colors.qualitative.Plotly[3],
        "count_FT": px.colors.qualitative.Plotly[4]
    },
)

newnames = {
    'count_RX': 'Success',
    'count_FS': 'Failure (Signal Strength)',
    "count_FI": "Failure (Interference)",
    "count_FW": "Failure (Invalid Receiver State)",
    "count_FT": "Failure (Invalid Sender State)"
}

fig.for_each_trace(lambda t: t.update(name=newnames[t.name], legendgroup=newnames[t.name], hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name])))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="center",
    x=.5
))

fig.write_image("receive_states_absolute_50_1000.pdf", width=1100, height=618)

# Interference plot of smaller communities

In [29]:
# Here we want to show the interference test, thus we only use the runs designed to generate interferences
means = counts \
    .groupby(["Nodes", "Mode", "Messages/Node"]) \
    .mean()

means = means[
    (means.index.get_level_values(0) == 100)
    & (
        (means.index.get_level_values(1) == "SF7, 125kHz, 222B")
        #| (means.index.get_level_values(1) == "SF7, 250kHz, 222B")
        | (means.index.get_level_values(1) == "SF9, 125kHz, 115B")
        | (means.index.get_level_values(1) == "SF12, 125kHz, 51B")
    )
]

In [40]:
# Fig: Interferences under load
fig = px.line(
    means,
    x=means.index.get_level_values(2),        # Messages/Node
    y=["Success", "Failure (Signal Strength)", "Failure (Interference)", "Failure (Invalid Receiver State)", "Failure (Invalid Sender State)"],
    facet_col=means.index.get_level_values(1),    # Mode
    category_orders={
        "facet_col": ["SF7, 125kHz, 222B", "SF9, 125kHz, 115B", "SF12, 125kHz, 51B"],
    },
    labels={
        "facet_col": "Configuration",
        "value": "%",
        "x": "Messages per Node",
        "variable": "Packet State"
    },
    color_discrete_map={
        "Success": px.colors.qualitative.Plotly[0],
        "Failure (Signal Strength)": px.colors.qualitative.Plotly[1],
        "Failure (Interference)": px.colors.qualitative.Plotly[2],
        "Failure (Invalid Receiver State)": px.colors.qualitative.Plotly[3],
        "Failure (Invalid Sender State)": px.colors.qualitative.Plotly[4]
    },
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="center",
    x=.5
))

fig.write_image("interferences.pdf", width=1100, height=450)

# Distance plot

In [41]:
# Fig: Distances depending on mode and event
# Only 10 messages/node and 500 nodes
smaller_count = sampled_df[
    (sampled_df["Messages/Node"].isin([10]))
    & (sampled_df["Mode"] != "SF9, 125kHz, 115B")
    & (sampled_df["Nodes"] == 500)
]

fig = px.box(
    smaller_count,
    x="Mode", 
    y="Distance",
    color="Event",
    category_orders={
        "Mode": ["SF7, 250kHz, 222B", "SF7, 125kHz, 222B", "SF7, 125kHz, 51B", "SF9, 125kHz, 51B", "SF12, 125kHz, 51B"],
        "Event": ["RX", "FS", "FI", "FW", "TX"],
    },
    labels={
        "Mode": "Configuration",
        "Distance": "Distance (m)",
        "Event": "Packet State"
    },
    color_discrete_map={
        "RX": px.colors.qualitative.Plotly[0],
        "FS": px.colors.qualitative.Plotly[1],
        "FI": px.colors.qualitative.Plotly[2],
        "FW": px.colors.qualitative.Plotly[3],
        "TX": px.colors.qualitative.Plotly[4]
    },
)

newnames = {
    'RX':'Success',
    'FS': 'Failure (Signal Strength)',
    "FI": "Failure (Interference)",
    "FW": "Failure (Invalid Receiver State)",
    "TX": ""
}

fig.for_each_trace(lambda t: t.update(name=newnames[t.name], legendgroup=newnames[t.name], hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name])))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="center",
        x=.5
    ),
    yaxis = {
     'tickmode': 'array',
     'tickvals': [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000],
     'ticktext': ["0", "1000", "2000", "3000", "4000", "5000", "6000", "7000", "8000", "9000", "10000", "11000"],
    },
)

fig.write_image("distances.pdf", width=1100, height=450)